In [1]:
import _pickle as pickle
from Components import Comment, Post, POSPost, POSComment
from collections import defaultdict
from tqdm import tqdm
import numpy as np

In [2]:
v1, v2 = [], []
for i in range(6):
    d1 = pickle.load(open("data/v1-{}".format(i), "rb"))
    d2 = pickle.load(open("data/v2-{}".format(i), "rb"))
    v1.append(d1)
    v2.append(d2)

In [3]:
post_counter = 0
full_list = []
for i in range(6):
    for j, p in enumerate(v1[i]):
        full_list.append((post_counter, p, v2[i][j]))
        post_counter += 1

In [4]:
post_counter

18570

In [5]:
def filter_func(tup):
    # returns True if POS Tag is one of allowed
    allowed_tags = ['Noun', 'Verb', 'Adjective']
    return tup[1] in allowed_tags

In [6]:
tf_by_post = []
idf_counter = defaultdict(int) # counts the number of documents a term appears in
for p_idx, p, pos_p in tqdm(full_list):
    filtered_pos = list(filter(filter_func, pos_p.body_pos))
    word_counter = defaultdict(int)
    for word, tag in filtered_pos:
        if word_counter[word] == 0:
            idf_counter[word] += 1
        word_counter[word] += 1
    
    tf_by_post.append(word_counter)

100%|██████████| 18570/18570 [00:00<00:00, 36754.92it/s]


In [7]:
top_term_number = 5000
all_used_terms = list(sorted(idf_counter.keys(), key=lambda x: idf_counter[x], reverse=True))[:top_term_number]

In [8]:
num_terms = len(all_used_terms)
num_docs = post_counter
print('number of terms: %d' % num_terms)
print('number of documents: %d' % num_docs)

number of terms: 5000
number of documents: 18570


In [9]:
tfidf_vectors = []
for p_idx, p, pos_p in tqdm(full_list):
    post_vector = np.zeros((num_terms,))
    for t_idx, term in enumerate(all_used_terms):
        tf_val = tf_by_post[p_idx][term]
        idf_val = np.log(num_docs/idf_counter[term])
        post_vector[t_idx] = tf_val * idf_val
    tfidf_vectors.append(post_vector[:])
    del post_vector

100%|██████████| 18570/18570 [01:54<00:00, 161.57it/s]


In [10]:
def cos_dist(vec1, vec2):
    size1 = np.linalg.norm(vec1)
    size2 = np.linalg.norm(vec2)
    inner_prod = np.sum(vec1 * vec2)
    if (size1 * size2) == 0:
        return 0
    else:
        return inner_prod / (size1 * size2)

In [11]:
# tfidf search implementation
search_idx = 10000
search_vec = tfidf_vectors[search_idx]
sim_info = []
for p_idx, p, pos_p in tqdm(full_list):
    if p_idx == search_idx: 
        continue
    dist_from_query = cos_dist(search_vec, tfidf_vectors[p_idx])
    sim_info.append((p_idx, dist_from_query))

100%|██████████| 18570/18570 [00:00<00:00, 39917.95it/s]


In [12]:
print('Org post-----')
print(full_list[search_idx][1].text)
print('Org post over-----')
for p_idx, d in sorted(sim_info, key=lambda x: x[1], reverse=True)[:5]:
    print(d)
    print(full_list[p_idx][2].body)
    print('^^^^^^^^^^^^^^^^^')

Org post-----
[중앙도서관 열람실 사용할때 자리배정좀...]
카이스트 학우분들! 중앙도서관 열람실 이용할땐 꼭 자리배정 후에 사용해주세요
키오스크까지 가는게 귀찮다면 KAIST Library 앱으로 간단히 자리배정도 가능하니 꼭 해주세요
빈 자리 배정 받고 가면 항상 사람이 차있어서 가끔은 기분 상하는 일이 꽤 있어요 ㅠㅠ 간단한 매너니 지켜주시면 감사하겠습니다
----------
#분실과_습득
https://talkyou.in/pages/KaDaejeon/posts/16856/
익명 아바타
Org post over-----
0.3300355692190233
자리 다 쓰고 가실 때 시간이 남았으면 꼭 좌석배정취소 누르고 가주세요
자리에 아무것도 없는데 230분 예약되어 있어서 띠용~하고 갑니다
^^^^^^^^^^^^^^^^^
0.32837372863941183
안녕하세요. 다소 황당한 일이 있어 제보합니다. 오늘 오후에 도서관에서 공부하고 있다가 자리에 짐을 두고 밥을 먹으러 나갔는데, 돌아와 보니 어떤 분이 제 짐을 한쪽으로 밀어두고 의자에 걸어둔 가방은 바닥에 내려놓고 공부를 하고 계시더라고요. 평소에는 꼭 자리배정을 하고 앉는데 오늘은 이전에 쓰던 사람이 취소를 안 해서 두시간정도 남아있길래 그냥 앉았습니다. 시간이 지난 걸 모르고 30분정도 자리를 비웠다 돌아왔는데 그 사이에 다른 분이 앉은 것을 보니 좀 어이가 없었습니다. 자리배정을 하지 않은 것이 제 실수라고 할 수도 있지만, 몇 시간씩 비운 것도 아니고 짐이 있는데 꼭 치우고 앉아야 하셨는지 모르겠습니다. 다행히 다른 자리가 비어 그냥 옮겼지만 기분이 좋지 않네요. 다른 분들도 식사하러 가실 때 꼭 자리배정이 되어 있는지 확인하고 가셔야 이런 일이 없을 것 같습니다.
^^^^^^^^^^^^^^^^^
0.3268247231186533
중앙 도서관이 새롭게 리모델링되었어요!!
1층에 열람실 자리표 뽑는 기계도 설치되었고, 배정시간도 4시간으로 넉넉한데
우리 모두 열람실 자리표를 뽑아서 쓰는

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [20]:
word_count_mat = np.zeros((num_docs, num_terms))
for p_idx, p, pos_p in tqdm(full_list):
    post_dict = tf_by_post[p_idx]
    for t_idx, term in enumerate(all_used_terms):
        word_count_mat[p_idx][t_idx] = post_dict[term]

100%|██████████| 18570/18570 [00:27<00:00, 685.80it/s]


In [27]:
lda = LatentDirichletAllocation(n_components=100, n_jobs=-1, verbose=1)

In [28]:
lda.fit(word_count_mat)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=100, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=1)

In [48]:
topic_idx = 12
topic_vec = lda.components_[topic_idx]
max_associated_topic = np.argmax(lda.components_, axis=0)
for t_idx, term in enumerate(all_used_terms):
    if max_associated_topic[t_idx] == topic_idx:
        print(term)

댓글
드리다
달다
남기다
가지
필요하다
페메
갖다
파다
계시
어
은동
궁동
사
색
메세지
필통
펜
노란색
섞이다
스티커
퍼센트
계산기
가나
기프티콘
주우
수영장
샤프
두운
맛집
전자회로
뚜레쥬르
펜슬
지퍼
볼펜
